Load and parse the Dataset (Run this first!)

Update the directory which contains all xlsx files

In [ ]:
import pandas as pd

from ParseData import parse_dataset
from utils.PropertyNames import ColumnNames as Cols

patient_data = parse_dataset("/home/meocakir/Documents/Datasets/Diabetes", silent=False)
patients = patient_data[Cols.patient].unique()

len(patient_data)

Filter prune benchmark Test

In [ ]:
from utils.PropertyNames import MethodOptions as Opts
from Benchmark import benchmark

params = {
    "k": 5,
    "risky_chars": {0, 1},
    "risk_threshold": 0.2,
    "prune": True,
    "prune_method": Opts.filter,
    "prune_threshold": 1,
    "weight_thresholds": [1, 3, 8],
    "value_ranges": [(0, 2), (2, 3), (3, float('inf'))],
    "max_steps": 6,
    "naive_threshold": 15
}

excluded = patients.copy().tolist()

#excluded.remove('P11')
#excluded.remove('P26')
benchmark(patient_data[patient_data[Cols.patient].isin(excluded)].copy(), start_time_range_hours=0,
          end_time_range_hours=1, **params)


Adaptive prune benchmark Test

In [ ]:
from utils.PropertyNames import MethodOptions as Opts
from Benchmark import benchmark

params = {
    "k": 6,
    "risky_chars": {0, 1},
    "risk_threshold": 0.2,
    "prune": True,
    "prune_method": Opts.adaptive,
    "prune_threshold": 1,
    "weight_thresholds": [1, 4, 10],
    "value_ranges": [(0, 2), (2, 3), (3, float('inf'))],
    "max_steps": 6,
    "naive_threshold": 15
}
excluded = patients.copy().tolist()

pd.set_option('display.float_format', '{:.4f}'.format)

_, _, result_df = benchmark(patient_data[patient_data[Cols.patient].isin(excluded)].copy(), start_time_range_hours=0,
          end_time_range_hours=1, **params)

Adaptive prune benchmark Test on 2 patients

Plot Probability Distribution

In [ ]:
from utils.VisualizationUtils import draw_histogram
from deBruijn.ProbabilityGraph import ProbabilityGraph
from utils.PropertyNames import MethodOptions as Opts
from utils.PropertyNames import ColumnNames as Cols

k = 4
risky_chars: None
params = {
    "prune": False,
    "prune_method": Opts.filter,
    "prune_threshold": 3,
    "max_steps": 3,
}

sequences = []
for p in patients:
    float_seq = patient_data[patient_data[Cols.patient] == p]
    float_seq = float_seq.sort_values(Cols.date, ascending=True)[Cols.char]
    sequences.append(float_seq)

probability_graph = ProbabilityGraph(sequences=sequences, k=k)

print(f"Resulting graph: {probability_graph}")

probability_model = probability_graph.get_probability_model(**params)

draw_histogram(list(probability_model.probability_dict.values()), "Node Probability Distribution", "Probability",
               "Count", bins=20)


Draw timeline of one of our models

In [ ]:
from utils.VisualizationUtils import draw_timeline
from utils.PropertyNames import ColumnNames as Cols
from utils.PropertyNames import MethodOptions as Opts

from Benchmark import add_alerts, add_target_column

naive_threshold = 15

params = {
    "k": 6,
    "risky_chars": {0, 1},
    "risk_threshold": 0.2,
    "prune": True,
    "prune_method": Opts.adaptive,
    "prune_threshold": 1,
    "weight_thresholds": [1, 4, 10],
    "value_ranges": [(0, 2), (2, 3), (3, float('inf'))],
    "max_steps": 6,
}

# Pick an alert model here
alert_to_plot = Cols.combined_alert_and

patient_data_with_alerts = add_target_column(patient_data)
patient_data_with_alerts = add_alerts(patient_data_with_alerts, naive_threshold, **params)

excluded = patients.copy().tolist()

for p in ["P1",'P21']:
    draw_timeline(
        patient_data_with_alerts[patient_data_with_alerts[Cols.patient] == p].sort_values(Cols.date, ascending=True), p,
        Cols.prob_alert, include_already_dangerous=False)
    draw_timeline(
        patient_data_with_alerts[patient_data_with_alerts[Cols.patient] == p].sort_values(Cols.date, ascending=True), p,
        Cols.combined_alert_and, include_already_dangerous=False)


Regenerate our model with alert this time for further analysis

In [ ]:
from utils.PropertyNames import ColumnNames as Cols
from utils.PropertyNames import MethodOptions as Opts

from Benchmark import add_alerts, add_target_column

naive_threshold = 15

params = {
    "k": 6,
    "risky_chars": {0, 1},
    "risk_threshold": 0.2,
    "prune": True,
    "prune_method": Opts.adaptive,
    "prune_threshold": 1,
    "weight_thresholds": [1, 4, 10],
    "value_ranges": [(0, 2), (2, 3), (3, float('inf'))],
    "max_steps": 6,
}

patient_data_with_alerts = add_target_column(patient_data)
patient_data_with_alerts = add_alerts(patient_data_with_alerts, naive_threshold, **params)


See missed warnings count

In [ ]:
import numpy as np

no_warning = 0
gave_warning = 0

for p in patients:
    df = patient_data[patient_data[Cols.patient] == p].copy()
    df = df.dropna(
        subset=[Cols.target, Cols.naive_alert, Cols.prob_alert, Cols.combined_alert_and, Cols.combined_alert_or])
    crossed_70 = (df[Cols.value] < 70) & (df[Cols.value].shift(1) >= 70)
    alert_true = df[Cols.combined_alert_and].shift(1).astype(bool)
    target_true = df[Cols.target].shift(1).astype(bool)

    gave_warning += np.sum(crossed_70 & alert_true & target_true)
    no_warning += np.sum(crossed_70 & ~alert_true & target_true)

print('gave_warning:', gave_warning)
print('no_warning:', no_warning)
print(gave_warning / (gave_warning + no_warning))


Calculate forecast times

In [ ]:
from utils.VisualizationUtils import draw_histogram

time_diff_list = []

pdict = dict()

for p in patients:
    df = patient_data[patient_data[Cols.patient] == p].copy().sort_values(Cols.date, ascending=True).reset_index(
        drop=True)
    df = df.dropna(subset=[Cols.target, Cols.naive_alert, Cols.prob_alert, Cols.combined_alert_and,
                           Cols.combined_alert_or]).reset_index(drop=True)  # Reset index after dropna
    crossed_70 = (df[Cols.value] < 70) & (df[Cols.value].shift(1) >= 70)

    for i in range(1, len(df)):
        # Check if value crosses below 70
        if crossed_70[i]:
            start_time = df.loc[i, Cols.date]
            # Iterate backwards to find the earliest point where both alert and target are true
            for j in range(i - 2, -1, -1):
                # Ignore point if value goes under 70 again
                if df.loc[j, Cols.value] < 70:
                    break
                # Check if both target and alert are true
                elif bool(df.loc[1, Cols.combined_alert_and]) is False or bool(df.loc[1, Cols.target]):
                    end_time = df.loc[j, Cols.date]
                    time_diff = start_time - end_time
                    time_diff_list.append(time_diff)
                    if p not in pdict.keys():
                        pdict[p] = 1
                    else:
                        pdict[p] += 1
                    break  # Found the required point, no need to check further

# Convert list of timedelta objects to desired format (e.g., total seconds)
time_diff_seconds = [td.total_seconds() for td in time_diff_list]
time_diff_minutes = [td / 60 for td in time_diff_seconds]
time_diff_minutes = [x for x in time_diff_minutes if x <= 60]

print(len(pdict), pdict)
print(time_diff_minutes)


Draw Timeline of Forecasts

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

def draw_timeline(event_times, threshold=15):
    counter = Counter(event_times)

    # Sort events
    sorted_events = sorted(counter.items())

    # Vertical spacing between events at the same time
    vertical_step = 0.2

    # Create timeline plot
    plt.figure(figsize=(15, 4))
    plt.axvline(x=0, color='red', linestyle='--', label='Hypoglycemia Event (t=0)')  # Line at t=0
    first_x_plotted = False  # Flag to track whether the first 'x' marker has been plotted

    for time, count in sorted_events:
        mirror_time = -time  # Multiply by -1 to mirror the timeline
        if count > threshold:
            # If count exceeds threshold, plot a single special marker
            plt.scatter(mirror_time, 0, marker='X', s=100, color='green')
            plt.annotate(f'{count}\nWarnings', (mirror_time, 0), textcoords="offset points", xytext=(0,10), ha='center')  # Bold text
        else:
            # Calculate starting vertical offset for this time event to center it
            vertical_offset = -(count - 1) * vertical_step / 2
            for _ in range(count):
                plt.scatter(mirror_time, vertical_offset, marker='x', s=50, color='blue', label='Warning Instance' if not first_x_plotted else "")
                vertical_offset += vertical_step
                first_x_plotted = True

    plt.xlabel('Time (minutes)')
    plt.title('Hypoglycemia Forecast Timeline Compilation')
    plt.yticks([])  # Hide y-axis
    plt.xticks(range(-35, 1, 1))  # Set x-ticks
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# Test function with sample data
draw_timeline(time_diff_minutes)


Draw ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc
import pickle

filtered_df = result_df[result_df[Cols.isDangerous] == False]

filtered_df = filtered_df[[Cols.combined_alert_and, Cols.target, "Probabilistic_Alert_Prob"]]

filtered_df = filtered_df.dropna()

y_true = filtered_df[Cols.target]
y_prob = filtered_df["Probabilistic_Alert_Prob"]


fpr, tpr, thresholds = roc_curve(list(y_true), list(y_prob))
roc_auc = auc(fpr, tpr)

data_to_save = {
    "roc_auc": roc_auc,
    "fpr": fpr,
    "tpr": tpr
}

# Saving the data to a binary file
with open("Data/roc_data.pkl", "wb") as file:
    pickle.dump(data_to_save, file)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()  

Make Stationary Test

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss

from ParseData import parse_dataset
from utils.PropertyNames import ColumnNames as Cols


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)


def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)


data = parse_dataset("/home/meocakir/Documents/Datasets/Diabetes", silent=False)
patients = data[Cols.patient].unique()

for patient in patients:
    patient_data = data[data[Cols.patient] == patient]
    patient_data = patient_data.sort_values(Cols.date, ascending=True)[Cols.value]
    print(f'++++++++++++++++++{patient}++++++++++++++++++')
    adf_test(patient_data)
    kpss_test(patient_data)
